In [43]:
import numpy as np
import pandas as pd
from tqdm import tqdm
tqdm.pandas()
from gensim.models import Word2Vec
import random

# Word2vec 활용 유사 가수 추천

### 데이터 불러오기
데이터 출처  
- [카카오 아레나 Melon Playlist Continuation](https://arena.kakao.com/c/8/data)

In [2]:
#플레이 리스트
playlist = pd.read_json('data/train.json')
#곡 정보
song_meta = pd.read_json('data/song_meta.json')

In [3]:
display(playlist.head(4))
display(song_meta.head(4))

,tags,id,plylst_title,songs,like_cnt,updt_date
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 139541, 351214, 650298, 531057, 205238, 706183, 127099,...",71,2013-12-19 18:36:19.000
1,"[추억, 회상]",10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 244277, 461062, 696302, 442765, 532114, 586541, 33389, ...",1,2014-12-02 16:19:42.000
2,"[까페, 잔잔한]",76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598, 233195, 666852, 686560, 556426, 142974, 331878, ...",17,2017-08-28 07:09:34.000
3,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스, 겨울왕국, 크리스마스송]",147456,크리스마스 분위기에 흠뻑 취하고 싶을때,"[394031, 195524, 540149, 287984, 440773, 100335, 556301, 655561, 534818, 695032, 516602, 521739,...",33,2019-12-05 15:15:18.000


,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id
0,[GN0901],20140512,불후의 명곡 - 7080 추억의 얄개시대 팝송베스트,2255639,[2727],Feelings,[GN0900],[Various Artists],0
1,"[GN1601, GN1606]",20080421,"Bach : Partitas Nos. 2, 3 & 4",376431,[29966],"Bach : Partita No. 4 In D Major, BWV 828 - II. Allemande",[GN1600],[Murray Perahia],1
2,[GN0901],20180518,Hit,4698747,[3361],Solsbury Hill (Remastered 2002),[GN0900],[Peter Gabriel],2
3,"[GN1102, GN1101]",20151016,Feeling Right (Everything Is Nice) (Feat. Popcaan & Wale),2644882,[838543],Feeling Right (Everything Is Nice) (Feat. Popcaan & Wale),[GN1100],[Matoma],3


> 곡을 부른 가수는 리스트 형태로 저장되어 있다. 

### 플레이리스트 artist 컬럼 추가

In [4]:
# 노래를 부른 가수를 가져올 dict를 만든다
# 노래 id가 key, 값은 그 노래를 부른 가수의 리스트
songid_to_artists = dict(zip(song_meta['id'],song_meta['artist_name_basket']))

# 각 플레이리스트에 포함된 노래의 가수를 set에 추가하여 리턴하는 함수
def get_artist(playlist):
    artists = set()
    for song in playlist:
        artists.update(songid_to_artists[song])
    return list(artists)

In [5]:
#플레이리스트 내 수록된 곡들의 가수를 모은 컬럼
playlist['artists'] = playlist['songs'].progress_apply(get_artist)

100%|███████████████████████████████████████████████████████████████████████| 115071/115071 [00:05<00:00, 19687.77it/s]


# word2vec 임베딩

- 단어의 의미를 다차원 공간에 벡터화하는 방법
![word2vec](https://wikidocs.net/images/page/22660/%EB%8B%A8%EC%96%B4.PNG)

- 플레이리스트는 대부분 의도를 가지고 만들어졌기 때문에   
  플레이리스트 내의 가수들을 단어로 보고 w2v으로 임베딩하면 연관되거나 유사한 가수끼리 모일 것으로 기대   
  &nbsp;
- 한 단어로 주변의 단어를 예측하는 skip-gram 방식 사용

In [6]:
playlist.iloc[ random.sample(range(len(playlist)), 10) ][['artists']]

,artists
56972,"[프리스타일, 우이경 (Woo Yi Kyung), 혜령, 뱅크, 소냐, M.Street (엠스트리트), 브라운아이드걸스, 빅마마, 휘, 이은미, 김경록 (V.O.S), 캔디..."
109414,"[AOA, 가인, 스텔라, EXID, 걸스데이, 효민, 라니아, 헬로비너스, 레인보우, 선미, 달샤벳, 포엘 (4L), 씨스타, 지연, 레인보우 블랙, 피에스타]"
59644,"[Sia, Taylor Swift, Pentatonix, Maroon 5, Owl City, Jessie J, Ariana Grande, Straight No Chaser,..."
57905,"[조이 (JOY), 가인, 에이프릴 (APRIL), SHINee (샤이니), 도영 (DOYOUNG), B1A4, XIA (준수), 애프터스쿨, 소녀시대-태티서 (Girls`..."
101011,"[2LSON, Pentatonix, 태일 (블락비), 폴 송, 쿠기 (Coogie), 박재범, OLNL (오르내림), ILLIONAIRE RECORDS, 다이나믹 듀오, 폴..."
44453,"[사이먼 도미닉, 주석, MC몽, 프라이머리, San E, 은지원, 크루셜스타 (Crucial Star), 리쌍, MC 스나이퍼, 로꼬]"
33404,"[프롬, 민채, 멜로망스, 로지피피, TETE, 랄라스윗 (lalasweet), 만쥬한봉지, 에디전 (뷰티핸섬), 유현 (YOOHYUN), 노리플라이 (no reply), ..."
97305,[신승훈]
83338,"[키스 더 뉴에이지, 박철진, 붐아카데미, 키즈팡팡, 소울 싱어즈, 말하기듣기, 최영환, 모닉, 플라스틱 피플, 돼지]"
15453,"[신해철, Alter Bridge, Dark New Day, 노브레인, DAVINK (다빈크), Otep, 여자친구 (GFRIEND), Arch Enemy, 김사랑, 터보,..."


In [7]:
#플레이리스트 내 가수 수의 최대값
playlist['artists'].apply(lambda x : len(x)).max()

250

In [34]:
%%time
#가수의 최대 수를 고려하여 window 크기는 200으로 설정했고  
# 벡터 사이즈는 100으로 설정함

w2v_model = Word2Vec(playlist['artists'].tolist(), min_count = 1, size = 100 , window = 200, sg = 1, seed=20)

Wall time: 10min 9s


In [35]:
# w2v_model.save("artist_w2v_new.model")
# w2v_model = Word2Vec.load("artist_w2v.model")

## 임베딩 확인

In [36]:
w2v_model.wv.most_similar(['TWICE (트와이스)'], topn = 10)

[('여자친구 (GFRIEND)', 0.920008659362793),
 ('오마이걸', 0.916660487651825),
 ('러블리즈', 0.8950859308242798),
 ('Red Velvet (레드벨벳)', 0.8837219476699829),
 ('모모랜드 (MOMOLAND)', 0.8731927871704102),
 ('Apink (에이핑크)', 0.8724551796913147),
 ('에이프릴 (APRIL)', 0.8555316925048828),
 ('언니쓰', 0.8523924350738525),
 ('세븐틴', 0.8472380638122559),
 ('EXID', 0.843825101852417)]

In [37]:
w2v_model.wv.most_similar(['에픽하이 (EPIK HIGH)'], topn = 10)

[('긱스 (Geeks)', 0.8807218074798584),
 ('리쌍', 0.876873254776001),
 ('버벌진트', 0.8625829219818115),
 ('다이나믹 듀오', 0.8477752208709717),
 ('윤하 (YOUNHA)', 0.8359357118606567),
 ('Supreme Team', 0.8324116468429565),
 ('MC몽', 0.8311997652053833),
 ('MC 스나이퍼', 0.8247138261795044),
 ('매드클라운', 0.8200557231903076),
 ('Tablo', 0.8194074630737305)]

In [40]:
w2v_model.wv.most_similar(['S.E.S.'], topn = 10)

[('신화', 0.946173906326294),
 ('god', 0.9072445631027222),
 ('룰라', 0.8881131410598755),
 ('카라', 0.8857624530792236),
 ('제이', 0.8712475299835205),
 ('슈', 0.8667564392089844),
 ('씨스타', 0.8627978563308716),
 ('바다 (BADA)', 0.8623074293136597),
 ('베이비 복스', 0.8581351637840271),
 ('박경림', 0.8577195405960083)]